In [40]:
import requests
from bs4 import BeautifulSoup as bs
import csv as csv

#Add website with https and no / at the end
website ="https://www.pagina12.com.ar"
csv_filename="pagina12_news.csv"

In [41]:
def is_available(e):
    if e:
        return e.text.strip()
    else:
        return None
def get_home_articles(url:str) -> list:
  article = requests.get(url)
  container = bs(article.content, "html.parser")
  body = container.find("div", class_="main-wrapper")
  article_list = {"list":[]}
  counter = 0
  for h2 in body.find_all("h2", class_="article-title"):
    item = {}
    if(is_available(h2.a)!=None):
      item["title"]  = h2.a.text
      item["url"]  = h2.a.get("href")
      article_list["list"].append(item)
  return article_list["list"]

#uncomment for testing
#get_home_articles(website)

[{'title': 'Represión para los despedidos del INTI\xa0',
  'url': 'https://www.pagina12.com.ar/748685-represion-para-los-despedidos-del-inti'},
 {'title': '"Conviven con las ratas"',
  'url': 'https://www.pagina12.com.ar/748605-casi-veinte-dias-presos-tras-la-caza-policial'},
 {'title': 'El Gobierno convirtió a Télam en una Agencia de Publicidad del Estado',
  'url': 'https://www.pagina12.com.ar/748665-el-gobierno-convirtio-a-telam-en-una-agencia-de-publicidad-d'},
 {'title': '🔴En vivo. La cotización del dólar este lunes',
  'url': 'https://www.pagina12.com.ar/748652-dolar-blue-dolar-hoy-a-cuanto-cotizan-el-lunes-1-de-julio'},
 {'title': '🔴En vivo. "Fase dos": Caputo y Bausili se reúnen con los bancos',
  'url': 'https://www.pagina12.com.ar/748644-la-agenda-de-javier-milei-entre-el-ajuste-y-el-pacto-de-mayo'},
 {'title': '¿Al Gobierno le alcanza con haberse quedado sin excusas?',
  'url': 'https://www.pagina12.com.ar/748504-al-gobierno-le-alcanza-con-haberse-quedado-sin-excusas'},
 {'t

In [42]:

class retrieve_news:
    def __init__(self, url:str):
        article = requests.get(url)
        self.soup = bs(article.content, "html.parser")
        self.body = self.get_body()        
        self.title = self.get_title()
        

    def get_body(self) -> list:
        body = self.soup.find(class_="article-main-content")
        if(body != None):
            content=""
            for p in body.find_all("p"):
                content = content + p.text
                content = content + "\n"     
            #return [p.text for p in body.find_all("p")]
            return content
        else:
            return None
    
    def get_title(self) -> str:
        body = self.soup.find(class_="article-main-content")
        if(body != None): 
            return self.soup.find("h1").text
        else:
            return None

In [43]:
articles = get_home_articles(website)
article = retrieve_news(articles[0]["url"])
row_list = []
#col_names = ["TITLE", "URL", "BODY","SECTION"]
for i in range(len(articles)):
    article = retrieve_news(articles[i]["url"])
    row = [article.title, articles[i]["url"], article.body, "home"]
    row_list.append(row)

In [44]:
filename = csv_filename
with open(filename, 'a+', newline='') as f:
    fields = ['TITLE', 'URL',"BODY","SECTION"]
    writer = csv.writer(f)

    #moving file pointer at the start of the file
    f.seek(0)
    existing_lines = csv.reader(f)
    
    #finding no of lines in the file
    count = 0
    counter = 0
    for line in existing_lines:
        count+=1
        break
    
    if count>0:
        for row_item in row_list:
            flag=0
            #moving file pointer to the start of the file
            f.seek(0)

            #checking if row_item[1] is present in the second  column (URL) of csv file
            for line in existing_lines:
                if row_item[1] in line[1]:
                    flag=1 

            #if row_item[1] is not found                   
            if flag==0:
                writer.writerow(row_item)
                counter += 1
                print(str(counter) + ": "+ row_item[1])
    #if file is empty write the dictionary contents into the csv
    else:
        for row_item in row_list:
            writer.writerow(row_item)

1: https://www.pagina12.com.ar/748685-represion-para-los-despedidos-del-inti
2: https://www.pagina12.com.ar/748682-oscar-parrilli-el-gobierno-de-milei-va-a-terminar-mal
